In [11]:
import os
import cv2
import numpy as np
def distance(v1,v2):
    return np.sqrt(((v1-v2)**2).sum())
def knn(train,test,k=5):
    dist=[]
    for i in range(train.shape[0]):
        X=train[i,:-1]
        Y=train[i,-1]
        d=distance(test,X)
        dist.append([d,Y])
    dist=sorted(dist)
    dist=dist[:k]#select k nearest points
    labels=np.array(dist)[:,-1]
    output=np.unique(labels,return_counts=True)
    index=np.argmax(output[1])
    return output[0][index]

In [27]:
cap=cv2.VideoCapture(-1)
#face detection
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
path='/home/drack/ML/Face Recognition/'
face_data=[]
labels=[]
class_id=0#labes for given file
names={}#mapping between id and name
#data prep

for fx in os.listdir(path):
    if fx.endswith('.npy'):
        print(fx)
        #create a mapping between class_id and names
        names[class_id]=fx[:-4]
        data_item=np.load(path+fx)
        print(" Data Item ",data_item)
        print(data_item.shape)
        print("file loaded")
        face_data.append(data_item)
        print("Face Data ",face_data)
    #create labels for the class
        target=class_id*np.ones((data_item.shape[0],))
        print("Target ",target)
        class_id+=1
        labels.append(target)
        print("Labels ",labels)
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))

trainset=np.concatenate((face_dataset,face_labels),axis=1)
print("Face dataset ",face_dataset)
print("FAce Labels ",face_labels)
print("Trainset ",trainset)

DracK.npy
 Data Item  [[209 213 214 ...  90  99 132]
 [253 253 250 ...  89 101 134]
 [222 222 224 ...  53  55  89]
 ...
 [ 88  86  93 ... 140 120 106]
 [ 85  84  88 ...  95  69  63]
 [ 88  86  93 ...  74  44  35]]
(19, 30000)
file loaded
Face Data  [array([[209, 213, 214, ...,  90,  99, 132],
       [253, 253, 250, ...,  89, 101, 134],
       [222, 222, 224, ...,  53,  55,  89],
       ...,
       [ 88,  86,  93, ..., 140, 120, 106],
       [ 85,  84,  88, ...,  95,  69,  63],
       [ 88,  86,  93, ...,  74,  44,  35]], dtype=uint8)]
Target  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Labels  [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])]
Yogi.npy
 Data Item  [[ 74 104 129 ... 109 100 101]
 [ 64  71  90 ... 117  96  97]
 [ 82 108 138 ... 122 110 112]
 ...
 [ 70  95 121 ...  55  61  76]
 [ 68  91 122 ...  62  60  74]
 [ 65  97 116 ... 211 199 201]]
(45, 30000)
file loaded
Face Data  [array([[209, 213, 214, ...,  90,  99, 132]

In [7]:
while True:
    ret,frame=cap.read()
    if ret==False:
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    for face in faces:
        x,y,w,h=face
    
    #get face region of interest
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        #predict label(out)
        out=knn(trainset,face_section.flatten())
        #display on screen the name and rectangle around it
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        
    cv2.imshow("Faces",frame)
                    
    key=cv2.waitKey(1)& 0XFF
    if key==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()